In [1]:
import tensorflow as tf
import os
import glob

In [2]:
def data_input_fn(filenames, batch_size=256, shuffle=False):
    
    def _parser(record):
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'image/encoded': tf.FixedLenFeature([], tf.string)
        }
        parsed_record = tf.parse_single_example(record, features)
        image = tf.decode_raw(parsed_record['image/encoded'], tf.float32)

        label = tf.cast(parsed_record['label'], tf.int32)

        return image, label
        
    def _input_fn():
        dataset = (tf.contrib.data.TFRecordDataset(filenames)
            .map(_parser))
        if shuffle:
            dataset = dataset.shuffle(buffer_size=10_000)

        dataset = dataset.repeat(None) # Infinite iterations: let experiment determine num_epochs
        dataset = dataset.batch(batch_size)
        
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()
        
        return features, labels
    return _input_fn

In [3]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(input_layer, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'
    
    x = tf.layers.conv2d(inputs=input_layer, filters=squeeze, kernel_size=(1, 1), padding='valid', name=s_id + sq1x1)
    x = tf.nn.relu(x, name=s_id + relu + sq1x1)
    
    left = tf.layers.conv2d(inputs=x, filters=expand, kernel_size=(1, 1), padding='valid', name=s_id + exp1x1)
    left = tf.nn.relu(left, name=s_id + relu + exp1x1)
    
    right = tf.layers.conv2d(inputs=left, filters=expand, kernel_size=(3, 3), padding='valid', name=s_id + exp3x3)
    right = tf.nn.relu(right, name=s_id + relu + exp3x3)
    
    x = tf.concat([left, right], axis=-1, name=s_id + 'concat')
    return x
    

In [4]:
def net_model_fn(features, labels, mode, params):
    """Model function for CNN."""

    is_training = mode == tf.estimator.ModeKeys.TRAIN
#     dropout_rate = params.dropout_rate if is_training else 0
    
    input_layer = tf.reshape(features, [-1, 100, 221, 7], name='input_reshape')
    x = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=(3, 3), strides=(2, 2), padding='valid', name='conv1')
    x = tf.nn.relu(x, name='relu_conv1')
    x = tf.layers.max_pooling2d(inputs=x, pool_size=(3, 3), strides=(2, 2), name='pool1')
    
    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = tf.layers.max_pooling2d(inputs=x, pool_size=(3, 3), strides=(2, 2), name='pool3')
    
    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = tf.layers.max_pooling2d(inputs=x, pool_size=(3, 3), strides=(2, 2), name='pool5')
    
    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = tf.nn.dropout(x, keep_prob=0.5, name='drop9')
    x = tf.layers.conv2d(inputs=x, kernel_size=(1, 1), padding='valid', name='conv10')
    x = tf.nn.relu(x, name='relu_conv10')
    logits = tf.reduce_mean(x, [1, 2])
    
    loss = None
    train_op = None

    # Calculate Loss if not predicting (for both TRAIN and EVAL modes)
    if mode != tf.estimator.ModeKeys.PREDICT:
        onehot_labels = tf.one_hot(indices=labels, depth=3)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
        
        correct_prediction = tf.equal(labels, tf.cast(tf.argmax(logits, 1), tf.int32))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            tf.summary.scalar('loss', loss)
            tf.summary.scalar('accuracy', accuracy)
        else:
            tf.summary.scalar('validation_loss', loss)
            tf.summary.scalar('validation_accuracy', accuracy)
    
    
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=params.learning_rate,
            optimizer='Adam')    

    # Generate Predictions
    predictions = {
      'classes': tf.argmax(input=logits, axis=1),
      'probabilities': tf.nn.softmax(logits, name='softmax_tensor')
    }
    
    eval_metric = {
        'accuracy': tf.contrib.metrics.streaming_accuracy(labels, tf.argmax(logits, 1))
    }

    return tf.estimator.EstimatorSpec(mode=mode,
                                     loss=loss,
                                     train_op=train_op,
                                     eval_metric_ops=eval_metric,
                                     predictions=predictions)

    
    

In [5]:
run_config = tf.contrib.learn.RunConfig(
    model_dir='/mnt/training', 
    save_checkpoints_steps=20, 
    save_summary_steps=20)

hparams = tf.contrib.training.HParams(
    learning_rate=0.001, 
    dropout_rate=0.4,
    data_directory=os.path.expanduser("/mnt/chr1"))

model_estimator = tf.estimator.Estimator(
    model_fn=net_model_fn, 
    config=run_config,
    params=hparams
)

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9db0ebe320>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 20, '_save_checkpoints_secs': None, '_session_config': None, '_save_checkpoints_steps': 20, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/mnt/training'}


In [6]:
train_batch_size = 256
train_steps = 100 * 221 * 7 #DATASETSIZEHERE // train_batch_size # len dataset // batch size
train_input_fn = data_input_fn(os.path.join(hparams.data_directory, 'train-*.tfrecords'), batch_size=train_batch_size)
eval_input_fn = data_input_fn(os.path.join(hparams.data_directory, 'validation.tfrecords'), batch_size=100)

experiment = tf.contrib.learn.Experiment(
    model_estimator,
    train_input_fn=train_input_fn,
    eval_input_fn=eval_input_fn,
    train_steps=train_steps
)

experiment.train_and_evaluate()

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


ValueError: Dimension 1 in both shapes must be equal, but are 24 and 22 for 'fire2/concat' (op: 'ConcatV2') with input shapes: [?,24,54,64], [?,22,52,64], [] and with computed input tensors: input[2] = <-1>.